In [44]:
#import libraries

import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
import hvplot
#import matplotlib
import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
from itertools import islice
#import json

In [19]:
#load .env file

load_dotenv()

True

In [20]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [21]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [22]:
# Request stock symbol from user. This code will work will all available tickers on alpacas.
# "META" ticker has been temporarily hard coded into the system for ease of use

# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")

stock_symbol = "META"

In [23]:
# Establish what time frame increment we will be looking at
# currently we are working with the daily chart, but the theory would be to 
# allow the user to identify what time scale they would like to trade on
# shorter time frame allows for "day trades" larger time scales are geared
# more towards longer term investments and swing trading.

time_frame = "1Day"

In [24]:
# Get user input for the start date in datetime format
# the start date has been hard coded for ease of programming and troubleshooting our code

# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2016-01-01", tz="America/New_York").isoformat()

In [25]:
# Get user input for the end date in datetime format
# the end date has been hard coded for ease of programming and troubleshooting our code

# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

In [9]:
# trading_days = end_date - start_date

# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [26]:
# Calls the stock data from the period of time desired in the desired time increment

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [27]:
# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44 entries, 2020-10-12 04:00:00+00:00 to 2020-12-11 05:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        44 non-null     float64
 1   high         44 non-null     float64
 2   low          44 non-null     float64
 3   trade_count  44 non-null     int64  
 4   open         44 non-null     float64
 5   volume       44 non-null     int64  
 6   vwap         44 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 2.8 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,292730,270.200,32999660,275.227035
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,181923,277.580,19505211,276.201124
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,151136,277.620,16590158,273.981850
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,182305,267.600,16330684,266.033730
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,157636,267.375,18430693,267.458035


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-12-07 05:00:00+00:00,285.58,288.4900,278.20,153281,279.190,14005842,285.751969
2020-12-08 05:00:00+00:00,283.40,286.4272,281.55,136454,286.010,11974657,283.705184
2020-12-09 05:00:00+00:00,277.92,287.6300,271.75,312698,283.660,26765687,278.642389
2020-12-10 05:00:00+00:00,277.12,278.7300,271.86,209825,275.535,21534833,276.018129
2020-12-11 05:00:00+00:00,273.55,276.4800,270.25,158853,274.530,15609459,273.330511


In [12]:
# Reset index and display the first 5
# this also prepares for use with plotly library/visualization

#stock_data_plot = stock_data.reset_index()

# clean up datafram by removing vwap, volume, trade count columns
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results

stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
stock_data_plot = stock_data.reset_index()

stock_data.head()

In [32]:
# Assign plot values to plotly

fig = go.Candlestick(x=stock_data_plot['timestamp'],
                open=stock_data_plot['open'],
                high=stock_data_plot['high'],
                low=stock_data_plot['low'],
                close=stock_data_plot['close'])


cand = go.Figure(data=[fig])

In [43]:
# Customize the figure (optional)

cand.update_layout(
    width=1200, height=800,
    title="META, OCT - DEC",
    yaxis_title='META Stock'
)


In [ ]:
# Add new data columns to DF for body size and candle size
# body size and direction is value of open - close. 
# candle size is the absolute value of low - high

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

# Display columns with added columns

stock_data.head()

In [ ]:
# initialize variables to be used in calculations
# identify the max body size and minimum candle size within the dataframe

max_body_size = stock_data["body_size"].max()
min_candle_size = stock_data["candle_size"].min()

# keeps track of the count which would initiate a buy or sell
# current count is initialized at 0 as the count is calculated based off of historical data

current_count = 0

# the threshold that would initiate a buy and or sell

threshold = 2

# adds a documenting column to the dataframe and assigns a flat value to all rows

stock_data["card_count"] = current_count

# identifies the number of rows to be used to break the for loop

number_of_rows = len(stock_data)

# print information up to date to identify data progress

# print(max_body_size)
# print(min_candle_size)
# print(previous_candle)
# print(stock_data.iloc[0][4])
# print(number_of_rows)
# stock_data.head()

In [ ]:
# ...::: working code :::...
# initialize variables and dataframe for tracking buys, sells, and profits

initial_investment = 10000

# initialize strategy tracking variables and add to stock_data df

stock_data["capital"] = initial_investment
stock_data["number_of_shares"] = 0
stock_data["investment"] = 0
stock_data["profits"] = 0

# variables that will be used to calculate profits/losses from buys/sells

capital = initial_investment
number_of_shares = 0
investment = 0
profits = 0

stock_data.head()

In [ ]:
# initialize variables and assign values based off of first row for calculations

previous_close = stock_data.iloc[0][0]
previous_high = stock_data.iloc[0][1]
previous_low = stock_data.iloc[0][2]
previous_open = stock_data.iloc[0][3]
previous_body_size = stock_data.iloc[0][4]
previous_candle_size = stock_data.iloc[0][5]

# begin for loop to start calculating each row's data to assign a +1 or -1
# to our count and add adjust our current/previous counts accordingly

for index, row in islice(stock_data.iterrows(), 1, None):

# create an if statement to make sure the for loop finishes at the end of the data appropriately  
    
    if index == (number_of_rows):
        break
    
# assign current day numbers to work with conditional statements

    current_close = row[0]
    current_high = row[1]
    current_low = row[2]
    current_open = row[3]
    current_body_size = row[4]
    current_candle_size = row[5]
    # capital = row[7]
    # number_of_shares = row[8]
    # investment = row[9]
    # profits = row[10]

   # begin of conditional statements. this portion identifies what values to assign
   # to each individal row's data and to determine whether a threshold of "sell" or "buy"
   # is reached. this portion can be adjusted/added to in order to take into consideration
   # additional bar patterns

    # bullish hammer
    if (current_body_size > 0) and (current_body_size*2) < (current_candle_size):
        current_count = current_count + 1
    # bearish hammer
    elif (current_body_size < 0) and ((np.absolute(current_body_size)*2) < (current_candle_size)):
        current_count = current_count - 1
    # bullish engulfing candle
    elif (current_body_size > 0) and (previous_body_size < 1) and (current_body_size > previous_body_size):
        current_count = current_count + 1
    # bearish engulfing candle
    elif (current_body_size < 0) and (previous_body_size > 1) and (np.absolute(current_body_size) > (previous_body_size)):
        current_count = current_count - 1
    # bullish doji candle
    #elif (previous_body_size < 0) and (np.absolute(current_body_size) < (current_close * max_body_size)):
    #    current_count = current_count + 1

    # contains the current count actual value to the desired "buy/sell" threshold

    if current_count > threshold:
        current_count = threshold
    elif current_count < -(threshold):
        current_count = -(threshold)

    # conditional statements to identify when to purchase or sell a stock

    if (current_count == threshold) and (number_of_shares == 0):
        number_of_shares = np.floor(capital / current_close)
        investment = number_of_shares * current_close
        capital = capital - investment
    elif (current_count == -threshold) and (number_of_shares != 0):
        profits = (number_of_shares * current_close) - investment
        capital = capital + investment + profits
        investment = 0
        number_of_shares = 0
    else:
        profits = (number_of_shares * current_close) - investment

        #stock_data.at[index,"investment"] = number_of_shares * current_close
        #profits = row[9] - investment
        #stock_data.at[index,"profits"] = profits

    stock_data.at[index,"number_of_shares"] = number_of_shares
    stock_data.at[index,"investment"] = investment
    stock_data.at[index,"profits"] = profits
    stock_data.at[index,"capital"] = capital
    # set variables to prepare to analyze the following row

    previous_close = row[0]
    previous_high = row[1]
    previous_low = row[2]
    previous_open = row[3]
    previous_body_size = row[4]
    previous_candle_size = row[5]

    # keep track of the current count within the dataframe. assign that
    # to the current working row
    
    stock_data.at[index,"card_count"] = current_count

if (number_of_shares == 0):
    total_profit = capital
elif (number_of_shares != 0):
    total_profit = (capital + investment + profits) - initial_investment

total_profit = round(total_profit,2)
percent_profit = (total_profit/initial_investment)*100

print(f"total profits for your autotrader are: ${total_profit} a {percent_profit}%")
    
display(stock_data.head(40))
display(stock_data.tail(40))



